# Accelerometer

In [6]:
# Set up
import hebi
from time import sleep
import math
import scipy.constants

lookup = hebi.Lookup()
#group = lookup.get_group_from_names(['X5-1'], ['X-00404'])
#group = lookup.get_group_from_names(['Wheel'], ['Right'])
group = lookup.get_group_from_names(['Wheel'], ['Right', 'Left'])

if group is None:
    print('Group not found: Did you forget to set the module family and names above?')
    exit(1)
    
for entry in lookup.entrylist:
    print(entry)

Family: Wheel Name: Left Mac Address: 54:10:EC:31:35:41
Family: Wheel Name: Right Mac Address: D8:80:39:A2:3A:66


In [7]:
# HEBIの加速度計
group.feedback_frequency = 5

# gravity
g = scipy.constants.g

def feedback_handler(group_fbk):
    #x = group_fbk.accelerometer.item(0, 0)
    yR = group_fbk.accelerometer.item(0, 1)
    #z = group_fbk.accelerometer.item(0, 2)
    yL = group_fbk.accelerometer.item(1, 1)
    '''
    if x <= -1.0:
        a = int(x)
    elif x > 1.0:
        a = int(x)
    else:
        a = 0
    '''
    
    sinR = float(yR / g) # x = g * sinθ
    if sinR >= 1:
        sinR = 1
    radR = math.asin(sinR) # θ= Arcsin
    degR = int(math.degrees(radR))
    
    sinL = float(yL / g) # x = g * sinθ
    if sinL >= 1:
        sinL = 1
    radL = math.asin(sinL) # θ= Arcsin
    degL = int(math.degrees(radL))
    
    print('degR = {0}, degL = {1}'.format(degR, degL))
    #print('Received feedback. Accelerometer: x = {0:.2f}, y = {1:.2f}, z = {2:.2f}, deg = {3}\n'.format(x, y, z,deg))

In [10]:
'''
# Add a callback to react to feedback received on a background thread.
# This function needs to accept 1 argument, `group_fbk`
group.add_feedback_handler(feedback_handler)

# Wait for 3 seconds and stop
sleep(3)
group.clear_feedback_handlers()
'''

while True:
    group.add_feedback_handler(feedback_handler)
    
    key = input("Do you want to stop? ")
    if key == "y":
        group.clear_feedback_handlers()
        break
    print(key)

degR = -2, degL = 4
degR = -2, degL = 5
degR = -2, degL = 5
degR = -2, degL = 4
degR = -2, degL = 5
degR = -2, degL = 5
degR = -2, degL = 6
degR = -2, degL = 5
degR = -2, degL = 4
degR = -2, degL = 1
degR = -2, degL = 0
degR = -2, degL = 4
degR = -2, degL = 3
degR = -2, degL = 3
degR = -2, degL = 3
degR = -2, degL = 3
degR = -2, degL = 3
degR = -2, degL = 3
degR = -2, degL = 3
degR = -2, degL = 3
degR = -2, degL = 3
degR = -2, degL = 3
degR = -2, degL = 5
degR = -2, degL = 9
degR = -2, degL = 17
degR = -2, degL = 21
degR = -2, degL = 28
degR = -2, degL = 31
degR = -2, degL = 36
degR = -2, degL = 34
degR = -2, degL = 46
degR = -2, degL = 47
degR = -2, degL = 47
degR = -2, degL = 47
degR = -2, degL = 49
degR = -2, degL = 51
degR = -2, degL = 50
degR = -2, degL = 51
degR = -2, degL = 50
degR = -2, degL = 50
degR = -2, degL = 50
degR = -2, degL = 50
degR = -2, degL = 51
degR = -2, degL = 50
degR = -2, degL = 50
degR = -2, degL = 50
degR = -2, degL = 50
degR = -2, degL = 48
degR = -2, degL 

# Rotation

In [8]:
# Set up
import hebi
from time import sleep
from math import pi
#import numpy as np

lookup = hebi.Lookup()
#group = lookup.get_group_from_names(['X5-1'], ['X-00404'])
#group = lookup.get_group_from_names(['Wheel'], ['Right'])
group = lookup.get_group_from_names(['Wheel'], ['Right', 'Left', 'IO_BOARD'])

if group is None:
    print('Group not found: Did you forget to set the module family and names above?')
    exit(1)

In [9]:
# Sets the command lifetime to 100 milliseconds
group.command_lifetime = 100

In [12]:
# Add a callback function to respond to feedback with a "infinity spring" command
def feedback_rotation(group_fbk):
    # rad/sec
    x = 0.2
    spring_constant = pi * x
    group_command = hebi.GroupCommand(group.size)
    #group_command.velocity = spring_constant #無限回転
    #group_command.velocity = [-spring_constant, spring_constant]  # go
    group_command.velocity = [spring_constant, -spring_constant, 0]  # go back
    group.send_command(group_command)

In [13]:
'''
group.add_feedback_handler(feedback_rotation)


# Control the robot at 100Hz for 20 seconds
group.feedback_frequency = 100.0
sleep(1)
group.clear_feedback_handlers()
'''

while True:
    group.feedback_frequency = 100.0
    group.add_feedback_handler(feedback_rotation)
    
    key = input("Do you want to stop? ")
    if key == "y":
        group.clear_feedback_handlers()
        break
    print(key)

Do you want to stop? y


In [7]:
group_command = hebi.GroupCommand(group.size)
print(group_command.velocity)

[nan nan nan]


# Accel + Rotation

In [12]:
# Set up
import hebi
from time import sleep
from math import pi
import scipy.constants

lookup = hebi.Lookup()
group = lookup.get_group_from_names(['X5-1'], ['X-00404'])
#group = lookup.get_group_from_names(['Wheel'], ['Right'])
#group = lookup.get_group_from_names(['Wheel'], ['Right', 'Light'])

if group is None:
    print('Group not found: Did you forget to set the module family and names above?')
    exit(1)

In [15]:
def feedback_handler(group_fbk):
    
    # Sets the command lifetime to 100 milliseconds
    group.command_lifetime = 100
    
    # gravity
    g = scipy.constants.g
    
    # 3-axis accel
    x = group_fbk.accelerometer.item(0, 0)
        
    sin = float(x / g) # x = g * sinθ
    if sin >= 1:
        sin = 1
    rad = math.asin(sin) # θ= Arcsin
    deg = int(math.degrees(rad))
    
    if abs(deg) > 5 and abs(deg) <= 30:
        speed = pi * deg / 10
    elif deg > 30 and deg <= 60:
        speed = pi * (60-deg) / 10
    elif deg < -30 and deg >= -60:
        speed = pi * (-60-deg) / 10
    else:
        speed = 0
    
    #print('deg = {0}, rad/sec = {1}'.format(deg, speed))
    
    group_command = hebi.GroupCommand(group.size)
    
    group_command.velocity = speed #無限回転
    group.send_command(group_command)

In [16]:
while True:
    
    # Sets the feedback frequency to 100 milliseconds
    group.feedback_frequency = 100
    
    group.add_feedback_handler(feedback_handler)
    
    key = input("Do you want to stop? ")
    if key == "y":
        group.clear_feedback_handlers()
        break
    print(key)

Do you want to stop? y


# Actuator & I/O Board

In [1]:
# Set up
import hebi
from time import sleep
import math
import scipy.constants
import numpy as np

lookup = hebi.Lookup()
#group = lookup.get_group_from_names(['Wheel'], ['Right'])
group = lookup.get_group_from_names(['Wheel', 'HEBI'], ['Right', 'IO_BOARD'])

if group is None:
    print('Group not found: Did you forget to set the module family and names above?')
    exit(1)

for entry in lookup.entrylist:
    print(entry)

group_fbk = hebi.GroupFeedback(group.size)

Family: HEBI Name: IO_BOARD Mac Address: D8:80:39:A1:A9:53
Family: Wheel Name: Right Mac Address: D8:80:39:5A:49:E1


In [15]:
def feedback_handler(group_fbk):
    
    # Container to the IO feedback
    io = group_fbk.io
    A1 = np.asarray(io.a.get_float(1))
    B1 = np.asarray(io.b.get_int(1))
    B2 = np.asarray(io.b.get_int(2))
    
    # Sets the command lifetime to 100 milliseconds
    group.command_lifetime = 100
    
    # gravity
    g = scipy.constants.g
    
    # 3-axis accel
    x = group_fbk.accelerometer.item(0, 0)
    
    # The number of command
    group_command = hebi.GroupCommand(group.size)
    
    # Sequence
    seq = 0
    
    # Sequence Control
    if(seq == 0):
        if(B1[1] == 1):
            seq = 1
        if(B2[1] == 1):
            seq = 2
    elif(seq == 1):
        if(B1[1] == 0):
            speed = pi * A1
        else:
            speed = -pi * A1
        
        group_command.velocity = speed #無限回転
        group.send_command(group_command)
        
        if(B2[1] == 1):
            seq = 2
    elif(seq == 2):
        seq = 0
        
    print('{0}'.format(seq))

In [ ]:
while True:
    
    # Sets the feedback frequency to 5 milliseconds
    group.feedback_frequency = 5
    
    group.add_feedback_handler(feedback_handler)
    

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [17]:
group.add_feedback_handler(feedback_handler)
group.feedback_frequency = 10

# Wait 10 seconds
sleep(10)

group.clear_feedback_handlers()

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
